# Question 1: Interpretation of Coefficients

𝑒𝑥𝑝𝑒𝑛𝑑𝑖𝑡𝑢𝑟𝑒=873+0.0012𝑎𝑛𝑛𝑢𝑎𝑙_𝑖𝑛𝑐𝑜𝑚𝑒+0.00002𝑎𝑛𝑛𝑢𝑎𝑙_𝑖𝑛𝑐𝑜𝑚𝑒2−223.57ℎ𝑎𝑣𝑒_𝑘𝑖𝑑𝑠


Our model shows that expenditure is the annual spending on recreation in US dollars. Our model shows that the bias term is 873 (represented by constant). For an increase in annual income, we see that the first person in household will spend $1.2 more on recereational speanding with an increase of $1000 in annual income. We see that the second coeffiecient shows a quadratic equation. That means, an additional 0.02 X annual income. Finally, the last coefficient shows that those with kids (our dummy variable) they will spend $223.57 less than the families without a child.

We do not the statistical significancy of the above coefficients.

# Question 2: Weather Model
1) Build a linear regression model where your target variable is the difference between the apparenttemperature and the temperature. As explanatory variables, use humidity and windspeed. Now, estimate your model using OLS. 
~Are the estimated coefficients statistically significant? 
~Are the signs of the estimated coefficients in line with your previous expectations? 
~Interpret the estimated coefficients. 
~What are the relations between the target and the explanatory variables?

2) Next, include the interaction of humidity and windspeed to the model above and estimate the model using OLS. Are the coefficients statistically significant? Did the signs of the estimated coefficients for humidity and windspeed change? Interpret the estimated coefficients.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'weatherinszeged'

In [2]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
weather_df = pd.read_sql_query('select * from weatherinszeged',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()

In [3]:
weather_df.head

<bound method NDFrame.head of                            date        summary preciptype  temperature  \
0     2006-03-31 22:00:00+00:00  Partly Cloudy       rain     9.472222   
1     2006-03-31 23:00:00+00:00  Partly Cloudy       rain     9.355556   
2     2006-04-01 00:00:00+00:00  Mostly Cloudy       rain     9.377778   
3     2006-04-01 01:00:00+00:00  Partly Cloudy       rain     8.288889   
4     2006-04-01 02:00:00+00:00  Mostly Cloudy       rain     8.755556   
...                         ...            ...        ...          ...   
96448 2016-09-09 17:00:00+00:00  Partly Cloudy       rain    26.016667   
96449 2016-09-09 18:00:00+00:00  Partly Cloudy       rain    24.583333   
96450 2016-09-09 19:00:00+00:00  Partly Cloudy       rain    22.038889   
96451 2016-09-09 20:00:00+00:00  Partly Cloudy       rain    21.522222   
96452 2016-09-09 21:00:00+00:00  Partly Cloudy       rain    20.438889   

       apparenttemperature  humidity  windspeed  windbearing  visibility  \
0    

In [4]:
weather_df.columns

Index(['date', 'summary', 'preciptype', 'temperature', 'apparenttemperature',
       'humidity', 'windspeed', 'windbearing', 'visibility', 'loudcover',
       'pressure', 'dailysummary'],
      dtype='object')

## 1: Build the OLS Model

In [6]:
#Use Humidity and Windspeed as feature set to explain temperature (Y-variable/Target)

Y = weather_df['apparenttemperature'] - weather_df['temperature']
X = weather_df[['humidity','windspeed']]

# We add a constant to the model as it's a best practice
# to do so every time!
X = sm.add_constant(X)

# We fit an OLS model using statsmodels
results = sm.OLS(Y, X).fit()

# We print the summary results
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                 1.949e+04
Date:                Tue, 28 Apr 2020   Prob (F-statistic):               0.00
Time:                        14:24:51   Log-Likelihood:            -1.7046e+05
No. Observations:               96453   AIC:                         3.409e+05
Df Residuals:                   96450   BIC:                         3.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.4381      0.021    115.948      0.0

We see that the coefficients for both 'humidity' and 'windspeed' are statistically signficiant with p-values less than .01. Both expalanatory variables are negatively correlated with the difference in apparent temperature and measured temperature. We may interpret the coefficients to mean that for each 1 point increase in humidity we see a drop in 3 units of temperature. For every one unit change in 'windspeed', we see a drop of .12 units of temperature. 

## 2: Calculate interaction of humidity and windspeed

In [7]:
# X interaction is the feature set
weather_df['humidity_windspeed_interaction'] = weather_df.humidity * weather_df.windspeed

# Y is the target variable
Y = weather_df['apparenttemperature'] - weather_df['temperature']


X = weather_df[['humidity','windspeed', 'humidity_windspeed_interaction']]

X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                 1.666e+04
Date:                Tue, 28 Apr 2020   Prob (F-statistic):               0.00
Time:                        14:32:44   Log-Likelihood:            -1.6669e+05
No. Observations:               96453   AIC:                         3.334e+05
Df Residuals:                   96449   BIC:                         3.334e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const                              0.0839      0.033      2.511      0.012       0.018       0.149
humidity                           0.1775      0.043      4.133      0.000       0.093       0.262
windspeed                          0.0905      0.002     36.797      0.000       0.086       0.095
humidity_windspeed_interaction    -0.2971      0.003    -88.470      0.000      -0.304      -0.291
==============================================================================
Omnibus:                     4849.937   Durbin-Watson:                   0.265
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9295.404
Skew:                          -0.378   Prob(JB):                         0.00
Kurtosis:                       4.320   Cond. No.                         193.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Interpretation: We captured non-linear relationships between the target temperature and the feature (interaction between humidity and windspeed). 

The new OLS Model is:

Temperature = .0839 + .178(humidity) + .090(windspeed) -.30(humidity_windspeed_interaction)

So we can interpret the above model as the following:
Given a windspeed level, 1 point increase in humidity results in 0.18 - 0.30 X windspeed point increase in the target. This means that the increase in the target (temperature) is lower for high values of windspeed than for low values of windspeed. So, the windspeed mitigates the effect of humidity increase on the target. Similarly for a given humidity level, 1 point increase in the windspeed results in 0.09 - 0.30 X humidity point increase in the target. So, the humidity also mitigates the effect of windspeed on the target.